# Problem set 2
### Ning Hua

## Q1 - use kallisto and reproduce Moriarty's result


In [1]:
!kallisto

kallisto 0.46.2

Usage: kallisto <CMD> [arguments] ..

Where <CMD> can be one of:

    index         Builds a kallisto index 
    quant         Runs the quantification algorithm 
    bus           Generate BUS files for single-cell data 
    pseudo        Runs the pseudoalignment step 
    merge         Merges several batch runs 
    h5dump        Converts HDF5-formatted results to plaintext
    inspect       Inspects and gives information about an index
    version       Prints version information
    cite          Prints citation information

Running kallisto <CMD> without arguments prints usage information for <CMD>



In [2]:
!kallisto index -i arc.idx arc.fasta.gz


[build] loading fasta file arc.fasta.gz
[build] k-mer length: 31
[build] counting k-mers ... done.
[build] building target de Bruijn graph ...  done 
[build] creating equivalence classes ...  done
[build] target de Bruijn graph has 19 contigs and contains 10000 k-mers 



In [3]:
! kallisto quant -i arc.idx -o arc_output --single -l 150 -s 20 arc.fastq.gz                  


[quant] fragment length distribution is truncated gaussian with mean = 150, sd = 20
[index] k-mer length: 31
[index] number of targets: 10
[index] number of k-mers: 10,000
[index] number of equivalence classes: 26
[quant] running in single-end mode
[quant] will process file 1: arc.fastq.gz
[quant] finding pseudoalignments for the reads ... done
[quant] processed 100,000 reads, 99,980 reads pseudoaligned
[   em] quantifying the abundances ... done
[   em] the Expectation-Maximization algorithm ran for 62 rounds



In [4]:
!ls arc_output/

abundance.h5  abundance.tsv run_info.json


In [5]:
!grep "^#" arc_output/abundance.tsv
!grep -v "^#" arc_output/abundance.tsv| sort -n -k5 

target_id	length	eff_length	est_counts	tpm
Arc6	3000	2851	1729.89	17073.6
Arc1	4000	3851	2608.95	19063.2
Arc9	3000	2851	2568.63	25351.7
Arc2	2000	1851	3626.07	55123
Arc4	4000	3851	10613.7	77552.6
Arc7	2000	1851	5497.17	83567.2
Arc8	2000	1851	5785.68	87953.1
Arc5	4000	3851	12635.9	92328.7
Arc10	3000	2851	26310.9	259682
Arc3	3000	2851	28603.1	282305


So the TPM results given by Kallisto is way closer to what Moriarty reported with a simple glimpse. And Arc6 and Arc1 are indeed more similar to Arc9 (talking about TPM, see table above). 

## Q2
### b. simulate an Arc transcriptome and RNA-seq reads - alternative (harder) version of part 2
Write Python code to simulate an Arc locus, an Arc transcriptome, and 100,000 reads from an RNA-seq experiment on the Arc transcriptome. Your script generates two simulated data files that kallisto will take as its input:

a FASTA format file of the transcriptome
a FASTQ format file of the reads
Your simulation will be controlled by several parameters. It'll be useful to leave them as settable parameters that you can play with in different simulations. For example, here's a chunk from mine:


In [6]:
# Set up the Arc locus 
## I made some changes to the names for easier reference
n_transcripts = 10                        # Number of segments in the Arc locus (A..J)
n_reads = 100000                          # total number of observed reads we generate
seg_len = 1000                            # length of each segment (nucleotides)
arc_len = seg_len * n_transcripts         # total length of the Arc locus (nucleotides)
read_len = 75                             # read length
basic_phases = "A C G T".split()
transcript_lengths = [4000, 2000, 3000, 4000, 4000, 
                      3000, 2000, 2000, 3000, 3000]  ## copied from the pset

Set up your simulation so that it can either use the structure of the Arc locus shown above (i.e. those specific lengths Li and those specific abundances νi), or it can generate new random choices for lengths of each transcript (in segments) Li and abundances νi.

To generate an Arc locus DNA sequence:
- assume S=10 segments, A-J
- assume each segment is 1000 bp long
- make random DNA of uniform base composition, 25% each base (ACGT)
- your total DNA sequence will be 10,000 bp

In [7]:
import numpy as np
import numpy.random as rand
rand.seed(42)

In [8]:
def generate_arc_loc(arc_len: int=arc_len):
    """Generates a DNA seqeunce, which is a list"""
    arc_locus = []
    ## random generation from base phases
    for i in range(arc_len):
        arc_locus.append(np.random.choice(basic_phases, p=[0.25,0.25,0.25,0.25]))
    
    return "".join(arc_locus)

To generate the Arc1...Arc10 mRNA transcripts:
- extract them from the Arc locus by their coordinates (being careful to handle the circular permutation!)
- write them in FASTA format to a .fasta output file.

In [9]:
def arc_to_transcriptome(arc_locus:str, transcript_lengths:list=transcript_lengths):
    """
    Given an arc lotus, returns a transcriptome
    By defult this function uses transcript_lengths provided in the pset
    """
    arc_transcripts = []
    seg_start = 0   ## tracking the start position of each transcript
    
    i=0
    for transcript_len in transcript_lengths:
        first_seg = arc_locus[seg_start:]  ## there might be a second part in a new circle 
        if seg_start + transcript_len > len(arc_locus):  ## if completes a circle
            arc_transcripts.append(first_seg + arc_locus[:(transcript_len-len(first_seg))])  ## find the second part in the new circle
        else:
            arc_transcripts.append(arc_locus[seg_start:seg_start+transcript_len])
        seg_start += seg_len
        if seg_start >= len(arc_locus):
            seg_start = 0

    return arc_transcripts

def find_num_lines(str_len, line_max_char):
    """
    returns the number of lines to fit a string given the max number of chars a line
    """
    if str_len%line_max_char == 0:
        return int(str_len/line_max_char)
    else:
        return int(str_len/line_max_char)+1
        
def write_transcriptome(arc_transcripts, output_file):
    """
    writes our transcripts to a fasta file 
    """
    line_max_char = 80
    with open(output_file, "w") as file:
        for transcript_num, transcript in enumerate(arc_transcripts):
            file.write(">"+"Arc"+str(transcript_num + 1)+"\n")
            ## split them to lines of 80 characters
            trans_l = [transcript[i*line_max_char:(i+1)*line_max_char] 
                       for i in range(find_num_lines(len(transcript), line_max_char))]  
            file.write("\n".join(trans_l)+"\n")
    file.close()

To generate reads, kallisto models the cDNA library fragment length distribution (so that it can calculate an "effective length" of each mRNA, correcting for the fact that library fragmentation and size selection selects against small cDNAs). So to generate each read, first have your simulation generate a random fragment, then generate a read from one of its ends:
- sample a transcript i according to its nucleotide abundance νi;
- pick a random fragment length at least as long as one read from a truncated Gaussian of mean length 150, standard deviation 20 (truncated because: no shorter than a read, and no longer than the whole transcript), i.e. something like:
```
while True:
    fraglen = int(np.random.normal(mean_frag, sd_frag))
    if fraglen >= len_R: break
if fraglen > L[i]: fraglen = L[i]
```
- pick a random fragment of that length from transcript i;
- pick a 75nt read by choosing a random orientation, taking the read from one of the two ends of the fragment. If on the top strand, your read is the first 75nt of the fragment. If on the bottom strand, your read is the last 75nt of the fragment, reverse complemented.
- generate a simulated read sequence by adding simulated base calling errors to the 75nt read: given base calling accuracy α, at each base, with probability (1−α), change it to one of the other 3 bases.
- output the 75nt read starting from that position in FASTQ format to your read file. Repeat for all 100K reads.

In this version, in your kallisto quant step, you'll give kallisto the same fragment length distribution parameters you used to simulate the data, i.e.:
```
kallisto quant -i <indexfile> -o <outputdir> --single -l 150 -s 20 <fastqfile>
```

In [10]:
default_abundance = [0.008, 0.039, 0.291, 0.112, 0.127, 
                     0.008, 0.059, 0.060, 0.022, 0.273] ## copied from the pset            

gaussian_mean = 150
gaussian_sd = 20
alpha = 0.999

In [11]:
comp_d = {"A":"T", "T":"A", "C":"G", "G":"C"}   ## complimentary 
def find_reverse_comp(seq):
    return "".join(list(map(lambda x: comp_d.get(x), seq[::-1])))  

find_reverse_comp("TTAGGGC")  

'GCCCTAA'

In [12]:
def mutate_with_base_error(seq:str, alpha=alpha):
    """mutates a sequence with a given base error alpha"""
    def mutate(c):
        return np.random.choice([bp for bp in basic_phases if bp!=c]) if rand.uniform(0,1) <= (1-alpha) else c
    return "".join(list(map(lambda x: mutate(x), seq)))
mutate_with_base_error("AAACCCGGGTTTTATATATA", 0.5)  ## test, if alpha=0.5, about half of the bases should mutate

'CCATACGCGTATTCGTGAAT'

In [13]:
def normalize(l):
    """
    normalizes a list of floats
    """
    return [i/sum(l) for i in l]

def gaussian_read_len(sample_max_len, sample_min_len=read_len, mean=gaussian_mean, sd=gaussian_sd):
    """
    pick a random fragment length at least as long as one read from a truncated Gaussian of mean length 150, 
    standard deviation 20 (truncated because: no shorter than a read, and no longer than the whole transcript)
    """
    while True:
        fraglen = int(np.random.normal(mean, sd))
        if fraglen >= sample_min_len: break
    if fraglen > sample_max_len: fraglen = sample_max_len
    return fraglen


In [14]:
## apparently my notebook is not configured properly
import sys
sys.path =  ['/opt/homebrew/anaconda3/envs/mcb/lib/python39.zip',
             '/opt/homebrew/anaconda3/envs/mcb/lib/python3.9',
             '/opt/homebrew/anaconda3/envs/mcb/lib/python3.9/lib-dynload',
             '/opt/homebrew/anaconda3/envs/mcb/lib/python3.9/site-packages']

In [15]:
from tqdm import trange

def transcripts_to_reads(transcripts:list, abundance:list=default_abundance):
    reads = []
    for i in trange(n_reads):
        ## sample based on aundance
        transcript_idx = np.random.choice(len(transcripts), p=normalize(abundance))
        transcript = transcripts[transcript_idx]  ## @param: str
        transcript_len = len(transcript)
        ## random read length that's shorter than the transcript length but longer than the default read_len
        frag_len = gaussian_read_len(sample_max_len=transcript_len, sample_min_len=read_len)
        start_idx = np.random.randint(0, (transcript_len-frag_len)+1)
        frag = transcript[start_idx:start_idx+frag_len]
        ## reverse complement w/ 1/2 probability
        if np.random.uniform(0,1) > 0.5:
            reads.append(mutate_with_base_error(frag[:75]))
        else:
            reads.append(mutate_with_base_error(find_reverse_comp(frag)))
    return reads

In [16]:
def write_reads(reads, output_file):
    with open(output_file, "w") as file:
        for i, read in enumerate(reads):
            file.write("@read%s\n"%str(i))
            file.write(read+"\n")
            file.write("+\n")
            file.write("I"*len(read)+"\n")

## 3. Test kallisto
Use your simulator (from 2a or 2b above) to generate dataset(s) using the lengths for the Arc locus as shown above, and the abundances that you think are true (first table, above). Analyze the simulated data with kallisto.

Does kallisto get the correct answer?

In [17]:
locus_simulation = generate_arc_loc(arc_len)

In [18]:
transcriptome_simulation = arc_to_transcriptome(locus_simulation)
write_transcriptome(transcriptome_simulation, "arc_simulation.fasta")
!head -10 arc_simulation.fasta

>Arc1
GCATGCAAAGGTACATCAGATCCTGCGGTTGGGTGCCAACCCAAGTGTGTTCACGGGCGCTTGACAGACATCGGAGGATG
GTGCACACTCACTCGACCAGCGCAAAGCACAGGATCTCACGGGCGGACATCTCTTAGGTCAGTCATCGTGGAGGAATGCT
TGTACGTTCTTTTGGCTTCCCCTAACACGGCGGGCGTCTCCGGTACGTATCCTGTCGGTACACCCCTTAAGCCCCTAGGC
CCGAAGAACATAGCGCATTTCACGCTCTCTACGAATGACCGCAACGATCAAATGGGCGAGAACAACTAATTCCGATTCAT
GGGGTTTGTGGATTGTGACACAGCGCGCCCGCTACTGCGGGACGTGAGGACGCCCAATTCTGCCAAGGATTATTTAGGGT
GTTTCACTAGAGTTATGCGCCGACCCCGGTTGGACCAGCTTGCATTCGAAACTGCGTTACACAGCACCCCACCGCAATCG
TATGACTCTCGCTGAAAAAGGGTGGTCAACCATTACACCTCTTATGCCTGTTGTGGGAGGCTCGGTCTTAAGCAGCGCGC
GAGCTGTGATCCAGGCTACCACGGACATAGTGTATGGAAAGTGATCCAGAGTAGACCCGCGGGGGCCTGACCTAACCTAT
ATAAGTTGTATCGTGGCTATGAGGGTAGTCGCCGGAGAAAACGTATGCTTACTGATTTTTAAGTCGGCGTGGCGCCGAAG


In [19]:
reads_simulation = transcripts_to_reads(transcripts=transcriptome_simulation)
reads_simulation[:5]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100000/100000 [00:14<00:00, 6767.98it/s]


['AGTCTGAGCAGTTCTTGATAGAGCCCAGTTAGGCAGTGCGATGATCTCACCAGGGAGAGATGGCACTTAGGGAAC',
 'GCTTAAGGTCTAGCAAGATCCTTAGGCCTATGAGGAAGGTTGTGAGTTTTAAATCCAGGGGTATAACCCCTACTACCACTGCTGCAATGTCTGCATACCTCGTCCGTCTTCCAGGTTTTTCTTACGTCGAACACTAAACCGCCACGCATGACACT',
 'TCCTACCCCACATCCTGATAATGCAGCTATGTGGCAATTCACGCTTACCCAATCCTTAGCTGGCTAACAATTCCC',
 'AGTTCTGGCGTCTGAACGTATTATTGTTGCTGGCTATCACAGTTAATTCCCTGCCTACGAATTTTGTCGTACCAA',
 'GGAATGATTCGTTGGTCGTTTTTTGTTTTCACGGTCCCGATGCTCCTCTATAGGATAATTAAGAGCAGATTAATGTCTAATATGTGAGCACTAGCGGCTCAGCGCACCAGGAAGCATTCTGAGTACACCCCCCCTCGGCGGATCCCAGAACTTTCGTGCCACA']

In [20]:
write_reads(reads_simulation, "arc_simulation.fastq")
!head -15 arc_simulation.fastq

@read0
AGTCTGAGCAGTTCTTGATAGAGCCCAGTTAGGCAGTGCGATGATCTCACCAGGGAGAGATGGCACTTAGGGAAC
+
IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII
@read1
GCTTAAGGTCTAGCAAGATCCTTAGGCCTATGAGGAAGGTTGTGAGTTTTAAATCCAGGGGTATAACCCCTACTACCACTGCTGCAATGTCTGCATACCTCGTCCGTCTTCCAGGTTTTTCTTACGTCGAACACTAAACCGCCACGCATGACACT
+
IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII
@read2
TCCTACCCCACATCCTGATAATGCAGCTATGTGGCAATTCACGCTTACCCAATCCTTAGCTGGCTAACAATTCCC
+
IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII
@read3
AGTTCTGGCGTCTGAACGTATTATTGTTGCTGGCTATCACAGTTAATTCCCTGCCTACGAATTTTGTCGTACCAA
+


In [21]:
!wc -l arc_simulation.fasta

     387 arc_simulation.fasta


In [22]:
!kallisto index -i arc_sim.idx arc_simulation.fasta
!kallisto quant -i arc_sim.idx -o arc_sim_output --single -l 150 -s 20 arc_simulation.fastq



[build] loading fasta file arc_simulation.fasta
[build] k-mer length: 31
[build] counting k-mers ... done.
[build] building target de Bruijn graph ...  done 
[build] creating equivalence classes ...  done
[build] target de Bruijn graph has 19 contigs and contains 10000 k-mers 


[quant] fragment length distribution is truncated gaussian with mean = 150, sd = 20
[index] k-mer length: 31
[index] number of targets: 10
[index] number of k-mers: 10,000
[index] number of equivalence classes: 26
[quant] running in single-end mode
[quant] will process file 1: arc_simulation.fastq
[quant] finding pseudoalignments for the reads ... done
[quant] processed 100,000 reads, 99,985 reads pseudoaligned
[   em] quantifying the abundances ... done
[   em] the Expectation-Maximization algorithm ran for 56 rounds



In [23]:
!grep "^#" arc_sim_output/abundance.tsv
!grep -v "^#" arc_sim_output/abundance.tsv| sort -n -k5

target_id	length	eff_length	est_counts	tpm
Arc6	3000	2851	2163.6	21382.4
Arc1	4000	3851	2928.33	21425.1
Arc9	3000	2851	2804.18	27713.1
Arc2	2000	1851	3312.08	50416.2
Arc4	4000	3851	10023.1	73334.2
Arc7	2000	1851	5382.42	81930.9
Arc8	2000	1851	5836.87	88848.5
Arc5	4000	3851	12655.6	92594.4
Arc10	3000	2851	26131.8	258255
Arc3	3000	2851	28747	284100


In [24]:
def calc_correct_tpm(abundance, lengths):
    return {f"Arc{idx+1}": round((abund/lengths[idx])/
                           sum(np.array(abundance)/np.array(lengths))*
                           (10**6), 1)
           for idx, abund in enumerate(abundance)}

correct_tpm = dict(sorted(calc_correct_tpm(default_abundance, transcript_lengths).items(), 
                    key=lambda item: item[1]))
correct_tpm

{'Arc1': 5904.1,
 'Arc6': 7872.1,
 'Arc9': 21648.2,
 'Arc2': 57564.6,
 'Arc4': 82656.8,
 'Arc7': 87084.9,
 'Arc8': 88560.9,
 'Arc5': 93726.9,
 'Arc10': 268634.7,
 'Arc3': 286346.9}

It's easy to see that Kallisto calculated some wrong TPM values. Although the ranks of **Arc1**, **Arc6**, and **Arc9** regarding TPM seem to be correspondent to Moriarty's Kallisto result, the excat values are off for quite a bit.

## 4. "debug" kallisto
Maybe you just found that kallisto isn't actually getting its estimation quite right.

Suggest a plausible reason that kallisto might be messing up the Arc analysis. What's most unusual about Arc, that might violate kallisto's assumptions somehow? Design at least one experiment that uses your simulator to test your hypothesis -- i.e., identify a modification that gives simulated data that kallisto works fine on.

### Answer
It's quite easy for us to think of the circularity as the reason for the inaccuracy - Too many overlaps shared by each transcript, which might ruin the performance of the underlying algorithm used by Kallisto. We could test this by running the same procedure on a ***"linear"*** locus simulation, which is (let's put it simple) longer than the total length of all tanscripts (so no overlaps). Also, let's change the way we mark the start point of each transcript to make there really no overlaps (```seg_start += transcript_len``` rather than ```seg_start += seg_len```).

In [25]:
def linear_arc_to_transcriptome(arc_locus:str, transcript_lengths:list=transcript_lengths):
    """
    Given an arc lotus, returns a transcriptome
    By defult this function uses transcript_lengths provided in the pset
    """
    arc_transcripts = []
    seg_start = 0   ## tracking the start position of each transcript
    
    i=0
    for transcript_len in transcript_lengths:
        first_seg = arc_locus[seg_start:]  ## there might be a second part in a new circle 
        if seg_start + transcript_len > len(arc_locus):  ## if completes a circle
            arc_transcripts.append(first_seg + arc_locus[:(transcript_len-len(first_seg))])  ## find the second part in the new circle
        else:
            arc_transcripts.append(arc_locus[seg_start:seg_start+transcript_len])
        seg_start += transcript_len


    return arc_transcripts


In [26]:
np.random.seed(1048)

## creat a giant locus so we won't run into overlaps
linear_locus = generate_arc_loc(arc_len=999999)

transcriptome_simulation = linear_arc_to_transcriptome(linear_locus)
write_transcriptome(transcriptome_simulation, "arc_simulation.fasta")
reads_simulation = transcripts_to_reads(transcripts=transcriptome_simulation)
write_reads(reads_simulation, "arc_simulation.fastq")
!kallisto index -i arc_sim.idx arc_simulation.fasta
!kallisto quant -i arc_sim.idx -o arc_sim_output --single -l 150 -s 20 arc_simulation.fastq
!grep "^#" arc_sim_output/abundance.tsv
!grep -v "^#" arc_sim_output/abundance.tsv| sort -n -k5

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100000/100000 [00:14<00:00, 6788.68it/s]



[build] loading fasta file arc_simulation.fasta
[build] k-mer length: 31
[build] counting k-mers ... done.
[build] building target de Bruijn graph ...  done 
[build] creating equivalence classes ...  done
[build] target de Bruijn graph has 10 contigs and contains 29700 k-mers 


[quant] fragment length distribution is truncated gaussian with mean = 150, sd = 20
[index] k-mer length: 31
[index] number of targets: 10
[index] number of k-mers: 29,700
[index] number of equivalence classes: 10
[quant] running in single-end mode
[quant] will process file 1: arc_simulation.fastq
[quant] finding pseudoalignments for the reads ... done
[quant] processed 100,000 reads, 99,685 reads pseudoaligned
[   em] quantifying the abundances ... done
[   em] the Expectation-Maximization algorithm ran for 52 rounds

target_id	length	eff_length	est_counts	tpm
Arc1	4000	3851	789	5738.96
Arc6	3000	2851	785	7712.63
Arc9	3000	2851	2095	20583.4
Arc2	2000	1851	3932	59502.7
Arc4	4000	3851	11221	81618.4
Arc7	2000	18

In [56]:
correct_tpm


{'Arc1': 5904.1,
 'Arc6': 7872.1,
 'Arc9': 21648.2,
 'Arc2': 57564.6,
 'Arc4': 82656.8,
 'Arc7': 87084.9,
 'Arc8': 88560.9,
 'Arc5': 93726.9,
 'Arc10': 268634.7,
 'Arc3': 286346.9}

The new results look much better and are very similar to the correct TPMs. We could've calculated an error table to compare the before-after results, but the differences are too obvious so I'll just leave it with eyeballing. 
